In [1]:
import os
import pandas as pd
from import_utils import ImportUtils
iu = ImportUtils(os.environ["LOCALDB"])

In [ ]:
url = "https://opendata.arcgis.com/datasets/7135e173c49145b18312163c1e27e3f0_0.zip?outSR=%7B%22latestWkid%22%3A2927%2C%22wkid%22%3A2927%7D"
iu.download_and_unzip_file(url)

In [ ]:
!shp2pgsql -s 2286 -W latin1 -D data/Tax_Parcels.shp import.pierce_county_parcel_geo > data/pierce_county_parcel_geo.sql
!psql -U postgres -d work -f data/pierce_county_parcel_geo.sql

In [ ]:
!psql -U postgres -d work --tuples-only -c "select 'DROP TYPE IF EXISTS pierce_county_landuse_type;  CREATE TYPE pierce_county_landuse_type AS ENUM (' || array_to_string(array_agg(quote_literal(dd) ORDER BY dd), ', ') || ');' from (select distinct landuse_de dd from import.pierce_county_parcel_geo) t" > sql/create_pierce_county_enums.sql
!psql -U postgres -d work -f sql/create_pierce_county_enums.sql

In [ ]:
import_utils.run_clean_script("pierce_county_parcel_geo")
import_utils.run_sql("DROP TABLE import.pierce_county_parcel_geo")
import_utils.run_command("rm data/Tax_Parcels.*")

In [ ]:
iu.download_and_unzip_file("https://online.co.pierce.wa.us/datamart/improvement.zip")
iu.download_and_unzip_file("https://online.co.pierce.wa.us/datamart/sale.zip")
iu.download_and_unzip_file("https://online.co.pierce.wa.us/datamart/improvement_builtas.zip")

In [ ]:
iu.run_sql(open("sql/pierce-county/create_sale.sql").read())
iu.run_sql(open("sql/pierce-county/create_improvement.sql").read())
iu.run_sql(open("sql/pierce-county/create_improvement_builtas.sql").read())

In [ ]:
iu.load_data_from_file("import.pierce_sale", "data/sale.txt", "|")
iu.load_data_from_file("import.pierce_improvement", "data/improvement.txt", "|")
iu.load_data_from_file("import.pierce_improvement_builtas", "data/improvement_builtas.txt", "|")

In [6]:
iu.load_data_from_file("import.pierce_sale", "data/sale.txt", "|")

[2022-11-06 21:58:59,894] {import_utils.py:21} INFO - b'COPY 589880\n'


In [ ]:
import_utils.run_command("rm data/improvement_builtas.txt")
import_utils.run_command("rm data/improvement.txt")
import_utils.run_command("rm data/sale.txt")

In [3]:
df = pd.read_csv("data/sale.txt", delimiter="|", encoding="Latin-1")

In [4]:
df.columns

Index(['%016403', '1', '5555514269', '06/29/2004', '5000.00',
       'Mobile Home Title Transfer', 'RANCHO VILLA MHC LLC',
       'FREDERICK DORAINE C', '0', '0.1', 'Unnamed: 10', '1.1', 'Mobile Home'],
      dtype='object')